In [105]:
%matplotlib notebook
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import plotly.express as px
from pylab import rcParams
from datetime import timedelta
import time
import warnings
warnings.filterwarnings('ignore')

# Read and Explore Data

## input

In [106]:
inputfile="C:/Users/changjin.feng/Desktop/dc/DC_UK.xlsx"
inputfile_NL="C:/Users/changjin.feng/Desktop/dc/DC_NL.xlsx"
dep_gate='LGW'
dest_gate='PMI'
rcParams['figure.figsize'] =40, 10
n_sim=1
ratio=2

In [107]:
data_cax=pd.read_excel(inputfile,sheet_name='S19 Flt CAX')
data_cax=data_cax.loc[(data_cax['GATEWAY_CODE_DEPARTURE']==dep_gate)&(data_cax['GATEWAY_CODE_ARRIVAL']==dest_gate),:]
data_cax.head(10)

,SEASON_YEAR,SEASON_TYPE,TRANSPORT_CODE,GATEWAY_CODE_DEPARTURE,GATEWAY_CODE_ARRIVAL,CYCLE_START_DATE,TRANSPORT_DEPARTURE_DATETIME,TRANSPORT_ARRIVAL_DATETIME,DIVISION_CODE,COMPANY_CODE,...,SOURCE_SYSTEM_ID,SOURCE_UPDATED_DATE,LOADED_DATE,CYRUS_TRS_ALLOCATION,FLIGHT_CARRIER,FLIGHT_FLAG,GUARANTEED_ALLOCATION,ALLOCATION_INDICATOR,RELEASE_DATE,VOL_DEAL_IND
72,2019,S,PMILGW6A,LGW,PMI,2019-10-12,2019-10-12 08:30:00,2019-10-12 11:45:00,G,NaN,...,A,10-OCT-19 15.03.39.000000000,11-OCT-19 00.54.58.825898000,325,NaN,NaN,NaN,NaN,NaN,NaN
73,2019,S,PMILGW6C,LGW,PMI,2019-06-08,2019-06-08 06:30:00,2019-06-08 09:25:00,G,NaN,...,A,06-JUN-19 18.42.00.000000000,07-JUN-19 00.49.56.377643000,325,NaN,NaN,NaN,NaN,NaN,NaN
74,2019,S,PMILGW6E,LGW,PMI,2019-07-13,2019-07-13 22:20:00,2019-07-14 01:35:00,G,NaN,...,A,13-JUL-19 12.03.24.000000000,14-JUL-19 00.43.54.174155000,189,NaN,NaN,NaN,NaN,NaN,NaN
105,2019,S,PMILGW3B,LGW,PMI,2019-07-24,2019-07-24 20:30:00,2019-07-24 23:45:00,G,NaN,...,A,24-JUL-19 19.03.15.000000000,25-JUL-19 00.41.47.138417000,221,NaN,NaN,NaN,NaN,NaN,NaN
106,2019,S,PMILGW3C,LGW,PMI,2019-05-08,2019-05-08 20:30:00,2019-05-08 23:45:00,G,NaN,...,A,07-MAY-19 13.03.36.000000000,08-MAY-19 01.45.42.364114000,221,NaN,NaN,NaN,NaN,NaN,NaN
107,2019,S,PMILGW3C,LGW,PMI,2019-08-14,2019-08-14 05:35:00,2019-08-14 09:00:00,G,NaN,...,A,08-AUG-19 14.03.46.000000000,09-AUG-19 00.46.49.324601000,221,NaN,NaN,NaN,NaN,NaN,NaN
108,2019,S,PMILGW3C,LGW,PMI,2019-09-04,2019-09-04 05:35:00,2019-09-04 09:00:00,G,NaN,...,A,03-SEP-19 00.23.41.000000000,04-SEP-19 00.47.40.527256000,221,NaN,NaN,NaN,NaN,NaN,NaN
109,2019,S,PMILGW5A,LGW,PMI,2019-08-30,2019-08-30 21:45:00,2019-08-31 01:00:00,G,NaN,...,A,30-AUG-19 18.03.26.000000000,31-AUG-19 00.49.59.732038000,221,NaN,NaN,NaN,NaN,NaN,NaN
110,2019,S,PMILGW6A,LGW,PMI,2019-06-01,2019-06-01 08:30:00,2019-06-01 11:45:00,G,NaN,...,A,31-MAY-19 18.03.02.000000000,01-JUN-19 00.46.22.638422000,305,NaN,NaN,NaN,NaN,NaN,NaN
111,2019,S,PMILGW6A,LGW,PMI,2019-08-31,2019-08-31 08:30:00,2019-08-31 11:45:00,G,NaN,...,A,28-AUG-19 00.23.50.000000000,29-AUG-19 00.45.17.748869000,305,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
data_booking_UK=pd.read_excel(inputfile,sheet_name='Historic Bookings')
data_booking_UK=data_booking_UK.loc[(data_booking_UK['GATEWAY_CODE_DEPARTURE']==dep_gate)&(data_booking_UK['GATEWAY_CODE_ARRIVAL']==dest_gate),:]
data_booking_UK=data_booking_UK.loc[data_booking_UK['CYCLE_START_DATE']>=data_booking_UK['BOOKING_DATE'],:]
data_booking_UK=data_booking_UK[['BOOKED_DURATION','CYCLE_START_DATE','BOOKING_DATE','PAX']]
data_booking_UK.head(10)

,BOOKED_DURATION,CYCLE_START_DATE,BOOKING_DATE,PAX
4062,7,2019-05-01,2018-07-15,4
4063,7,2019-05-01,2018-07-27,4
4064,7,2019-05-01,2018-08-27,2
4065,7,2019-05-01,2018-09-21,2
4066,7,2019-05-01,2018-10-15,2
4067,7,2019-05-01,2018-10-16,2
4068,7,2019-05-01,2018-10-17,2
4069,7,2019-05-01,2018-10-28,4
4070,7,2019-05-01,2018-11-02,2
4071,7,2019-05-01,2018-12-30,3


In [109]:
data_booking_NL=pd.read_excel(inputfile_NL,sheet_name='data')
data_booking_NL.columns=['Traject','CYCLE_START_DATE','BOOKING_DATE','PAX','BOOKED_DURATION']
data_booking_NL=data_booking_NL.loc[data_booking_NL['CYCLE_START_DATE']>=data_booking_NL['BOOKING_DATE'],:]
data_booking_NL['BOOKED_DURATION']-=1
data_booking_NL.head(10)

,Traject,CYCLE_START_DATE,BOOKING_DATE,PAX,BOOKED_DURATION
0,AMS-PMI,2018-05-11,2017-07-21,5,7
1,AMS-PMI,2018-05-08,2017-07-23,2,8
2,AMS-PMI,2018-05-02,2017-07-25,3,7
3,AMS-PMI,2018-05-21,2017-07-26,7,7
4,AMS-PMI,2018-06-15,2017-07-26,2,14
5,AMS-PMI,2018-08-25,2017-07-27,2,11
6,AMS-PMI,2018-06-16,2017-07-28,9,7
7,AMS-PMI,2018-04-15,2017-07-29,3,7
8,AMS-PMI,2018-05-09,2017-07-30,13,7
9,AMS-PMI,2018-05-02,2017-07-31,5,7


# Functions

## Preprocessing Distributions

* df0_perc: book month distribution
* df1_perc: departure week distribution per book month
* df2_perc: duration distribution per book month & departure week
* df3_perc: duration distribution per departure week (helper for df2_perc)

In [110]:
def preprocessing (data_booking): 
    
    # filter out negative bookings
    data_booking=data_booking.loc[data_booking['PAX']>0,:]
    
    # filter out duration langer than 29 nights
    data_booking=data_booking.loc[data_booking['BOOKED_DURATION']<30,:]
    
    data_booking['DEP_YEAR']=data_booking['CYCLE_START_DATE'].dt.year
    data_booking['BOOKING_YEAR']=data_booking['BOOKING_DATE'].dt.year
    data_booking['DIFF_YEAR']=data_booking['DEP_YEAR']-data_booking['BOOKING_YEAR']
    data_booking=data_booking.loc[data_booking['DIFF_YEAR']<=1,:]
    data_booking['YEAR_STR']='TY'
    data_booking['YEAR_STR'].loc[data_booking['DIFF_YEAR']==1]='LY'
    data_booking['BOOKING_YEAR_MONTH']=data_booking['YEAR_STR']+data_booking['BOOKING_DATE'].dt.strftime('%m')
    data_booking['DEP_WEEK']=data_booking['CYCLE_START_DATE'].dt.week
    data_booking=data_booking.loc[(data_booking['YEAR_STR']=='TY')|(data_booking['BOOKING_DATE'].dt.month>=4)]
    
        
    df0=data_booking.groupby(['BOOKING_YEAR_MONTH'],as_index=True)['PAX'].sum()
    df0_perc=df0.div(df0.sum())
    
    df1=pd.pivot_table(data_booking,values='PAX',index='BOOKING_YEAR_MONTH',columns='DEP_WEEK',aggfunc='sum')
    df1=df1.fillna(0)
    #df1.insert(0,df1.columns[0]-1,df1.iloc[:,0])
    df1_perc=df1.div(df1.sum(axis=1),axis=0)
    
    df2=pd.pivot_table(data_booking,values='PAX',index=['DEP_WEEK','BOOKING_YEAR_MONTH'],columns='BOOKED_DURATION',aggfunc='sum')
    df2=df2.fillna(0)
    df2sum=df2.sum(axis=1)
    df2_perc=df2.div(df2.sum(axis=1),axis=0)
    
    df3=pd.pivot_table(data_booking,values='PAX',index=['DEP_WEEK'],columns='BOOKED_DURATION',aggfunc='sum')
    df3=df3.fillna(0)
    df3_perc=df3.div(df3.sum(axis=1),axis=0)
    
    for i in range(len(df2_perc)):
        if df2sum.iloc[i]<10:
            df2_perc.iloc[i,:]=df3_perc.loc[df3_perc.index==df2_perc.index[i][0],:].values[0]
            
    df2_perc=df2_perc.div(df2_perc.sum(axis=1),axis=0)
    
    df_cax_remain=np.append(df_cax.to_numpy(copy=True),np.zeros([30,4]),axis=0)
    
    return df0_perc,df1_perc,df2_perc,df_cax_remain,data_booking

## Update Capacity

In [111]:
def update_cx(df0_perc,df1_perc,df2_perc,dw,dur,bm,dw_p,i,df_cax_remain,result):
    dw[i]=np.random.choice(df1_perc.columns,p=dw_p.iloc[i,1:])
    dw_bm=(dw[i],bm[i])
    dur[i]=np.random.choice(df2_perc.columns,p=df2_perc.iloc[[x for x in df2_perc.index.tolist()].index(dw_bm),:])
    if dur[i]>0:
        df_cax_sub=df_cax.loc[df_cax['WEEK']==dw[i],:]
        if len(df_cax_sub)>0:
            outbound_range=np.arange(df_cax_sub.index[0],df_cax_sub.index[-1]+1)
            bl=(df_cax_remain[outbound_range,1]>0) & (df_cax_remain[outbound_range+dur[i],2]>0)
            df_cax_sub=df_cax_sub.loc[bl,:]
            if len(df_cax_sub)>0:
                dp_date_index=np.random.choice(df_cax_sub.index,p=np.repeat(1/len(df_cax_sub),len(df_cax_sub)))
                df_cax_remain[dp_date_index,1]-=1
                df_cax_remain[dp_date_index+dur[i],2]-=1
                result.append([i,1,bm[i],dw[i],dur[i],'Accepted!'])
            else: result.append([i,1,bm[i],dw[i],dur[i],'Rejected: within departure week, duration is not available!'])
        else: result.append([i,1,bm[i],dw[i],dur[i],'Rejected: departure week is not available!'])
    else: result.append([i,1,bm[i],dw[i],dur[i],'Rejected: duration is 0!'])
    return df_cax_remain,result

## Plot 

In [112]:
def comparision_plot(result_sim,result_true,col_sim,col_true):
    result_sim.sort_values(by=col_sim,inplace=True)
    sim_temp=result_sim.groupby(col_sim,as_index=True)['npax'].sum()
    sim_perc=sim_temp.div(sim_temp.sum())
    
    result_true=result_true.loc[result_true['DEP_YEAR']==2019,:]
    #result_true=result_true.loc[result_true['PAX']>0,:]
    #result_true=result_true.loc[result_true['BOOKED_DURATION']<30,:]
    temp=result_true.groupby(col_true,as_index=True)['PAX'].sum()
    perc=temp.div(temp.sum())
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Simulation '+col_sim,'True '+col_true))
    fig.add_trace(go.Bar(x=sim_perc.index,y=sim_perc), 
          row=1, col=1)
    fig.add_trace(go.Bar(x=perc.index,y=perc), 
          row=1, col=2)
    fig.update_layout(height=600, width=1000, title_text="Comparision_Plots")
    fig.show()

# Simulation

In [113]:
def Simulation (df0_perc,df1_perc,df2_perc,n_sim):
    lf=[None]*n_sim
    result_final=pd.DataFrame()
    for sim_iter in range(0,n_sim):
        # book month random draw
        bm=np.random.choice(a=df0_perc.index,size=ttl_pax_per_round,p=df0_perc)
        
        dw=[None]*ttl_pax_per_round
        dur=[None]*ttl_pax_per_round
            
        dw_p=pd.merge(pd.DataFrame(bm,columns=['bm']),df1_perc,right_on=df1_perc.index,left_on='bm',how='left')

        df_cax_remain=np.append(df_cax.to_numpy(copy=True),np.zeros([30,4]),axis=0)

        result=[]
        
        #fig, ax = plt.subplots()
        #y_max=1200
        
        for i in range(0,round(len(bm))):
            
            #ani_bm=pd.DataFrame([bm[i]]*len(df_cax_remain0),columns=['book_month'])
            df_cax_remain,result=update_cx(df0_perc,df1_perc,df2_perc,dw,dur,bm,dw_p,i,df_cax_remain,result)
            #df_temp=pd.DataFrame(df_cax_remain[:-30,:],columns=['dep_date','cax_o','cax_i','week'])
            
            #if (i%100==0)|(i==round(len(bm))):
             #   ax.clear()
              #  ax.set_ylim((0,y_max))
               # ax.set_autoscaley_on(False)
                #ax.bar(df_temp['dep_date'], df_temp['cax_o'], color='blue')
                #print(sum(df_temp['cax_o']))
                #fig.canvas.draw()
            #plt.pause(0.001)
        #print(1-sum(df_cax_remain0[:,1]/sum(df_cax['CAX_O'])))
 
        lf[sim_iter]=1-sum(df_cax_remain[:,1]/sum(df_cax['CAX_O']))

        result=pd.DataFrame(result)
        result_final=pd.concat([result_final,result], ignore_index=True)
    return result_final,lf,df_cax_remain

# Capacity UK S19

In [114]:
df_cax=data_cax[['TRANSPORT_ARRIVAL_DATETIME','TRANSPORT_ALLOCATION']]
df_cax['TRANSPORT_ARRIVAL_DATETIME']=df_cax['TRANSPORT_ARRIVAL_DATETIME'].dt.date
df_cax=df_cax.groupby(['TRANSPORT_ARRIVAL_DATETIME'],as_index=False)['TRANSPORT_ALLOCATION'].sum()
daterange=pd.DataFrame(pd.date_range(start=min(df_cax['TRANSPORT_ARRIVAL_DATETIME']), end=max(df_cax['TRANSPORT_ARRIVAL_DATETIME'])).to_pydatetime().tolist(),columns=['TRANSPORT_ARRIVAL_DATETIME'])
daterange['TRANSPORT_ARRIVAL_DATETIME']=daterange['TRANSPORT_ARRIVAL_DATETIME'].dt.date
df_cax=pd.merge(daterange,df_cax,on='TRANSPORT_ARRIVAL_DATETIME',how='left')
df_cax=df_cax[['TRANSPORT_ARRIVAL_DATETIME','TRANSPORT_ALLOCATION']]
df_cax=df_cax.fillna(0)
df_cax['CAX_I']=df_cax['TRANSPORT_ALLOCATION']
df_cax['WEEK']=pd.to_datetime(df_cax['TRANSPORT_ARRIVAL_DATETIME']).dt.week
df_cax.columns=['DP_DATE','CAX_O','CAX_I','WEEK']

ttl_pax_per_round=round(sum(df_cax['CAX_O'])*ratio)
print('TTL Pax Per Round: '+str(ttl_pax_per_round))

TTL Pax Per Round: 100528


# Simulation Based on UK Booking Data

In [115]:
df0_perc_UK,df1_perc_UK,df2_perc_UK,df_cax_remain_UK,data_booking_UK=preprocessing(data_booking_UK)

In [116]:
plt.rcParams['figure.figsize'] = (5,6)
t=time.time()
result_final_UK,lf_UK,df_cax_remain_UK=Simulation (df0_perc_UK,df1_perc_UK,df2_perc_UK,n_sim)
df_cax_remain_UK=pd.DataFrame(df_cax_remain_UK[:-30,:],columns=df_cax.columns)
print(time.time()-t)

179.0694923400879


In [117]:
fig = make_subplots(rows=2, cols=2,shared_yaxes=True,subplot_titles=('Outbound Original Cax',
                                                                     'Outbound Free Cax',
                                                                    'Inbound Original Cax',
                                                                    'Inbound Free Cax'))
fig.add_trace(
    go.Bar(x=df_cax['DP_DATE'], y=df_cax['CAX_O'],marker=dict(color="green")),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=df_cax['DP_DATE'], y=df_cax['CAX_I'],marker=dict(color="green")),
    row=2, col=1
)

fig.add_trace(
    go.Bar(x=df_cax['DP_DATE'], y=df_cax_remain_UK['CAX_O'],marker=dict(color="magenta")),
    row=1, col=2
)

fig.add_trace(
    go.Bar(x=df_cax['DP_DATE'], y=df_cax_remain_UK['CAX_I'],marker=dict(color="magenta")),
    row=2, col=2
)
fig.update_yaxes(autorange="reversed", row=2, col=1)
fig.update_yaxes(autorange="reversed", row=2, col=2)


fig.update_layout(height=600, width=1000, title_text="Capacity before and after simulation",showlegend=False)
fig.show(),

(None,)

In [118]:
result_final_UK.columns=['id','npax','book_month','departure_week','duration','message']
result_final_UK.to_csv("C:/Users/changjin.feng/Desktop/dc/result_UK_without_DC.csv",index=False)
accepted_UK=result_final_UK.loc[result_final_UK['message'].str.contains('Accepted!', regex=True),:]

In [119]:
comparision_plot(result_final_UK,data_booking_UK,'book_month','BOOKING_YEAR_MONTH')

In [120]:
comparision_plot(accepted_UK,data_booking_UK,'book_month','BOOKING_YEAR_MONTH')

In [121]:
comparision_plot(result_final_UK,data_booking_UK,'departure_week','DEP_WEEK')

In [122]:
comparision_plot(accepted_UK,data_booking_UK,'departure_week','DEP_WEEK')

In [123]:
comparision_plot(result_final_UK,data_booking_UK,'duration','BOOKED_DURATION')

In [124]:
comparision_plot(accepted_UK,data_booking_UK,'duration','BOOKED_DURATION')

In [125]:
lf_UK[0]

0.9137354766831132

In [126]:
cax=df_cax_remain_UK.copy()
sum(cax['CAX_O'])

4336.0

# LF if all 7nights = LF ceiling without DC and one ways

In [127]:
o=df_cax['CAX_O'].iloc[:-7].values
i=df_cax['CAX_I'].iloc[7:].values
mtv=np.minimum(o,i)
sum(mtv)/sum(o)

0.9458341738955013